In [20]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
plt.ioff();
try:
    maindir
except NameError:
    maindir = os.path.dirname(os.getcwd())
for module in tqdm(os.listdir(maindir + '\\testing_modules')):
    %run -i "{maindir}\\testing_modules\\{module}"
base = os.path.dirname(os.path.dirname(maindir)) + '\\'
plt.ion();
np.set_printoptions(suppress=True)
plt.close('All')

100%|██████████| 20/20 [00:00<00:00, 193.43it/s]


In [21]:
%matplotlib qt
import skimage.io as io
import pyFAI
import h5py
from skimage.restoration import rolling_ball

In [22]:
scanid = 149248
scanid = 149566
filedir = '''D:\\Musterman_postdoc\\20231109_Musterman\\'''
#filedir = '''D:\\Musterman_postdoc\\20231030_Musterman\\'''
filedir = '''C:\\Users\\emusterma\\OneDrive - Brookhaven National Laboratory\\Documents\\Postdoc\\Data\\Condensed Datasets\\'''
map_filename = f'scan{scanid}_dexela_xrd.tif'
dtype = np.int16

test = XRDMap.from_image_stack(map_filename, wd=filedir, energy=15)
test.scanid = 149566
#test.map.estimate_background(method='rolling_ball', radius=10)
# Bad approximation
dark_field = XRDMap.from_image_stack(f'scan149567_dexela_xrd.tif', wd=filedir).map.max_image

AttributeError: 'XRDMap' object has no attribute '_h5'

In [15]:
filename = f'scan{test.scanid}_XRD_map.h5'
wd = filedir

f = h5py.File(f'{wd}{filename}', 'a') # 'a' allows for appending data
base_grp = f.require_group('xrdmap')
base_grp.attrs['scanid'] = test.scanid
base_grp.attrs['beamline'] = '5-ID (SRX)'
base_grp.attrs['facility'] = 'NSLS-II'
base_grp.attrs['energy'] = test.energy
base_grp.attrs['wavelength'] = test.wavelength
base_grp.attrs['time_stamp'] = '' # Not sure why I cannot assign None

# Record diffraction data
curr_grp = base_grp.require_group('image_data') # naming the group after the detector may be a bad idea...
curr_grp.attrs['detector'] = '' #'dexela'
curr_grp.attrs['detector_binning'] = '' #(4, 4)
curr_grp.attrs['exposure_time'] = '' # 0.1
curr_grp.attrs['expsure_time_units'] = 's'

# Raw Images
dset = curr_grp.require_dataset('raw_images', data=test.map.images,
                                shape=test.map.images.shape,
                                dtype=test.map.dtype,
                                compression="gzip", compression_opts=8) # add compression to more places
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'counts'
dset.attrs['dtype'] = str(test.map.dtype)
dset.attrs['time_stamp'] = ttime.ctime()

In [16]:
f.close()
test._h5.close()
test.map._h5.close()

In [10]:
filename = f'scan{test.scanid}_XRD_map.h5'
wd = filedir

f = h5py.File(f'{wd}{filename}', 'a') # 'a' allows for appending data
base_grp = f.require_group('xrdmap')
base_grp.attrs['scanid'] = test.scanid
base_grp.attrs['beamline'] = '5-ID (SRX)'
base_grp.attrs['facility'] = 'NSLS-II'
base_grp.attrs['energy'] = test.energy
base_grp.attrs['wavelength'] = test.wavelemgth
base_grp.attrs['time_stamp'] = None

# Record diffraction data
curr_grp = base_grp.require_group('image_data') # naming the group after the detector may be a bad idea...
curr_grp.attrs['detector'] = None #'dexela'
curr_grp.attrs['detector_binning'] = None #(4, 4)
curr_grp.attrs['exposure_time'] = None # 0.1
curr_grp.attrs['expsure_time_units'] = 's'

# Raw Images
dset = curr_grp.require_dataset('raw_images', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'counts'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

'''# Dark-field Image(s)
dset = curr_grp.require_dataset('dark_field', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'counts'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

# Dark-field Corrected Images
dset = curr_grp.require_dataset('dark_corrected_images', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'a.u.'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

# Flat-field Image(s)
dset = curr_grp.require_dataset('flat_field', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y'] # May change depending on image size
dset.attrs['units'] = 'a.u.'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

# Flat-field Corrected Images
# Do I really need to record this?
dset = curr_grp.require_dataset('flat_corrected_images', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'a.u.'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()'''

dset = curr_grp.require_dataset('processed_images', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'img_x', 'img_y']
dset.attrs['units'] = 'a.u.'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()
dset.attrs['corrections'] = ['dark-field', 'flat-field']
dset.attrs['correction_methods'] = [None, None] # Median, rolling ball, spline, etc.

# Calibrated Images
dset = curr_grp.require_dataset('calibrated_images', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'tth_ind', 'chi_ind']
dset.attrs['units'] = 'a.u'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

# Calibration Mask
dset = curr_grp.require_dataset('calibration_mask', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'tth_ind', 'chi_ind']
dset.attrs['units'] = 'a.u'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()

######################################################
# Blobs
curr_grp = base_grp.require_group('analysis')
dset = curr_grp.require_dataset('blobs', data=)
dset.attrs['labels'] = ['x_ind', 'y_ind', 'blobs']
dset.attrs['units'] = 'arbitrary'
dset.attrs['time_stamp'] = ttime.ctime()

# Phase
dset = curr_grp.require_dataset('phase', data=)
set.attrs['possible_phases'] = [phase.name for phase in test.phases]

# Orientation

# Strain

# Size/Shape

######################################################
# Record positiion data
curr_grp = base_grp.require_group('positions')
#f['/xrdmap/positions'].create_dataset('name', data=['x_pos', 'y_pos'])
#f['/xrdmap/positions'].create_dataset('pos', data=)
dset.attrs['labels'] = ['x_pos', 'y_pos']
dset.attrs['units'] = 'μm'
dset.attrs['dtype'] = np.uint16

# Record calibrated reciprocal space data
curr_grp = base_grp.require_group('recipriocal_positions')
#f['/xrdmap/positions'].create_dataset('name', data=['x_pos', 'y_pos'])
#f['/xrdmap/positions'].create_dataset('pos', data=)
dset.attrs['labels'] = ['tth', 'chi']
dset.attrs['comments'] = """'tth' is the two_theta scattering angle\n
                         'chi' is the azimuthal angle"""
dset.attrs['units'] = '° [deg.]'
dset.attrs['dtype'] = np.uint16
dset.attrs['time_stamp'] = ttime.ctime()
dset.attrs['calibration'] = None # Read poni file as dictionary
dset.attrs['extent'] = test.extent

# Record scalar data
curr_grp = base_grp.require_group('scalars')
dset.attr['labels'] = ['i0', 'i0_time', 'time', 'im', 'it'] # Not sure if I will need all of these...
dset.attr['units'] = 'counts'
dset.attr['dtype'] = np.uint16
#f['/xrdmap/scalers'].create_dataset('name', data=)
#f['/xrdmap/scalers'].create_dataset('val', data=)


f.close()

In [ ]:
test.map.subtract_background(background = dark_field)
flat_field = rolling_ball(median_filter(test.map.med_image, size=20), radius=10)

new_image_lst = []
for image in test.map.images.reshape(test.map.num_images, *test.map.image_shape):
    new_image = image / flat_field / np.median(image)
    new_image_lst.append(new_image)

test.map.images = np.asarray(new_image_lst).reshape(*test.map.shape)
test.map.dtype = np.float32

#test.map.subtract_background(background = background)
test.set_calibration('scan149562_dexela_calibration.poni')
#test.set_calibration('scan149256_dexela.poni')

In [13]:
import pyUSID

In [25]:
fname = 'scan2D_149248_xs_sum8ch.h5'

f = h5py.File(f'{wd}{fname}', 'r')

In [65]:
f['/xrfmap/scalers/name'].attrs['names'] = ['i0', 'i0_time', 'time', 'im', 'it']

RuntimeError: Unable to create attribute (no write intent on file)

In [16]:
pyUSID.hdf_utils.print_tree(f)

/
├ xrfmap
  ------
  ├ detsum
    ------
    ├ counts
    ├ xrf_fit
    ├ xrf_fit_name
  ├ positions
    ---------
    ├ name
    ├ pos
  ├ scalers
    -------
    ├ name
    ├ val
  ├ scan_metadata
    -------------
